In [19]:
import os

In [26]:
%pwd

'd:\\KC\\End to End MLflow\\research'

In [27]:
os.chdir('../.')

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)

class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path


In [29]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [31]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Ensure correct indentation here
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-18 10:20:19,992:INFO: common:yaml file: config\config.yaml loaded successfully]
[2025-03-18 10:20:19,996:INFO: common:yaml file: params.yaml loaded successfully]
[2025-03-18 10:20:19,998:INFO: common:yaml file: schema.yaml loaded successfully]


[2025-03-18 10:20:20,002:INFO: common:created directory at: artifacts]
[2025-03-18 10:20:20,002:INFO: common:created directory at: artifacts/data_ingestion]
[2025-03-18 10:20:21,100:INFO: 832124254:artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 187C:23CBE2:3B198B:9FEFA2:67D8FB8C
Accept-Ranges: bytes
Date: Tue, 18 Mar 2025 04:50:21 GMT
Via: 1.1 varnish
X-Served-By: cache-del21720-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1742273421.859680,VS0,VE506
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastl